# 1. Lightgbm

* 데이터 '와인품질분류.zip'입니다.

1. 필요한 라이브러리를 가져옵니다.

In [206]:
import pandas as pd
from lightgbm import LGBMClassifier

2. 데이터를 불러옵니다.

In [207]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [208]:
train = pd.read_csv('/content/drive/MyDrive/ESAA_2024-2/DataSets/wine_train.csv')
test = pd.read_csv('/content/drive/MyDrive/ESAA_2024-2/DataSets/wine_test.csv')
smpl_sub = pd.read_csv('/content/drive/MyDrive/ESAA_2024-2/DataSets/wine_sample_submission.csv')

In [209]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [210]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


3. 변수를 변환합니다.

In [211]:
# type에는 white와 red 두 종류가 있습니다.
# 각각 0,1로 변환합니다.
import numpy as np
train['type'] = train['type'].map({'white': 0, 'red': 1})
test['type'] = test['type'].map({'white': 0, 'red': 1})

In [212]:
train.head()

,index,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0
1,1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1
2,2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0
3,3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0
4,4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0


In [213]:
test.head()

,index,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0
1,1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1
2,2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0
3,3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0
4,4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0


In [214]:
# 모델에 입력하기 전 데이터를 정형합니다.
# train_x : train의 'index', 'quality' 피처를 제거하고 저장합니다.
# train_y : train의 'quality'를 저장합니다.
# test_x : test의 'index'를 제거하고 저장합니다.
train_x = train.drop(['index', 'quality'], axis=1)
train_y = train['quality']
test_x = test.drop(['index'], axis=1)

In [215]:
train_x.shape, train_y.shape, test_x.shape

((5497, 12), (5497,), (1000, 12))

In [216]:
train_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,0
1,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,1
2,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,0
3,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,0
4,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,0


In [217]:
train_y.head()

,quality
0,5
1,5
2,5
3,6
4,6


In [218]:
test_x.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,0
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,1
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,0
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,0
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,0


4. LightGBM 모델을 생성 및 훈련합니다.

In [219]:
lgbm = LGBMClassifier()

5. 생성한 모델로 예측 데이터를 생성합니다.

In [220]:
lgbm.fit(train_x, train_y)

LGBMClassifier()

6. 제출파일을 생성합니다.

In [221]:
#서브미션 파일의 quality 변수에 예측값을 대입합니다.
smpl_sub['quality'] = lgbm.predict(test_x)

In [222]:
smpl_sub.head()

,index,quality
0,0,5
1,1,5
2,2,6
3,3,5
4,4,6


In [223]:
# csv 파일로 저장합니다.
smpl_sub.to_csv('submission.csv', index=False)

# 2. XGBoost

In [224]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2024-10-18 12:12:47--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2024-10-18 12:12:47--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.251.2.100, 142.251.2.101, 142.251.2.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2024-10-18 12:12:47--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251

In [225]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from xgboost import XGBClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index', axis = 1, inplace =True)
test.drop('index', axis = 1, inplace =True)

In [226]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [227]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [228]:
# 원핫 인코딩 (pd.get_dummies())
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

encoded_type_train = encoder.fit_transform(train[['type']])
type_train = pd.DataFrame(encoded_type_train, columns=encoder.get_feature_names_out(['type']))
train = pd.concat([train.drop('type', axis=1), type_train], axis=1)

encoded_type_test = encoder.fit_transform(test[['type']])
type_test = pd.DataFrame(encoded_type_test, columns=encoder.get_feature_names_out(['type']))
test = pd.concat([test.drop('type', axis=1), type_test], axis=1)

In [229]:
# 모델 정의
xgb_clf = XGBClassifier()

In [230]:
# 모델 학습
# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수

X = train.drop('quality', axis=1)
y = train['quality'] -3

# fit 메소드를 이용해 모델 학습
xgb_clf.fit(X , y)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [231]:
# predict 메소드와 원핫 인코딩한 test 데이터를 이용해 품질 예측
pred_xgb = xgb_clf.predict(test)

In [232]:
# sample_submission.csv 파일을 불러와 예측된 값으로 채워 주기
sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA_2024-2/DataSets/wine_sample_submission.csv')
sample = sample_submission.copy()
sample['pred_xgb'] = pred_xgb + 3
sample.head()

,index,quality,pred_xgb
0,0,0,5
1,1,0,5
2,2,0,5
3,3,0,5
4,4,0,6


In [233]:
# 정답파일 내보내기
sample.to_csv('pred_xgb.csv',index = False)

# 3. 보팅

In [234]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2024-10-18 12:12:51--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2024-10-18 12:12:51--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 142.251.2.100, 142.251.2.101, 142.251.2.102, ...
Connecting to drive.google.com (drive.google.com)|142.251.2.100|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download [following]
--2024-10-18 12:12:51--  https://drive.usercontent.google.com/download?id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251

In [235]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [236]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [237]:
test.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,9.0,0.31,0.48,6.6,0.043,11.0,73.0,0.99380,2.90,0.38,11.6,white
1,13.3,0.43,0.58,1.9,0.070,15.0,40.0,1.00040,3.06,0.49,9.0,red
2,6.5,0.28,0.27,5.2,0.040,44.0,179.0,0.99480,3.19,0.69,9.4,white
3,7.2,0.15,0.39,1.8,0.043,21.0,159.0,0.99480,3.52,0.47,10.0,white
4,6.8,0.26,0.26,2.0,0.019,23.5,72.0,0.99041,3.16,0.47,11.8,white


In [238]:
# 원핫 인코딩 (pd.get_dummies())
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False)

encoded_type_train = encoder.fit_transform(train[['type']])
type_train = pd.DataFrame(encoded_type_train, columns=encoder.get_feature_names_out(['type']))
train = pd.concat([train.drop('type', axis=1), type_train], axis=1)

encoded_type_test = encoder.fit_transform(test[['type']])
type_test = pd.DataFrame(encoded_type_test, columns=encoder.get_feature_names_out(['type']))
test = pd.concat([test.drop('type', axis=1), type_test], axis=1)

In [239]:
# 모델 정의 (LightGBM, XGBoost, Random Forest)
lgbm_clf = LGBMClassifier()
xgb_clf = XGBClassifier()
rf_clf = RandomForestClassifier()

# VotingClassifier 정의
voting = VotingClassifier(estimators=[('lgbm', lgbm_clf), ('xgb', xgb_clf), ('rf', rf_clf)], voting='soft')

In [240]:
# 모델 학습

# X 는 train에서 quality 를 제외한 모든 변수
# y 는 train의 qulity 변수
X = train.drop('quality',axis = 1)
y = train['quality']

# fit 메소드를 이용해 모델 학습
voting.fit(X, y)

VotingClassifier(estimators=[('lgbm', LGBMClassifier()),
                             ('xgb',
                              XGBClassifier(base_score=None, booster=None,
                                            callbacks=None,
                                            colsample_bylevel=None,
                                            colsample_bynode=None,
                                            colsample_bytree=None, device=None,
                                            early_stopping_rounds=None,
                                            enable_categorical=False,
                                            eval_metric=None,
                                            feature_types=None, gamma=None,
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=None, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=None, ...)),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [241]:
# predict 메소드와 원핫 인코딩한 테스트 데이터를 이용해 품질 예측
pred_voting = voting.predict(test)

In [242]:
sample['pred_voting'] = pred_voting
sample.head()

,index,quality,pred_xgb,pred_voting
0,0,0,5,5
1,1,0,5,5
2,2,0,5,6
3,3,0,5,5
4,4,0,6,6
